In [ ]:
import cv2
import numpy as np

import sys
sys.path.append('/cephfs/SZ-AI/usr/liuchenyu/HaiLook/Moore-AnimateAnyone')
sys.path.append('/cephfs/SZ-AI/usr/liuchenyu/HaiLook/Moore-AnimateAnyone/tools')

from src.utils.util import get_fps, read_frames, save_videos_from_pil

from src.dwpose import DWposeDetector, draw_pose
from controlnet_aux.util import HWC3, resize_image
from PIL import Image
import matplotlib.pyplot as plt
from openpose_rescaler import draw, rescale_skeleton, crop_and_resize
from src.dataset.dance_video_aug import HumanDanceVideoAugDataset
import openpose_rescaler
import importlib
from tqdm import tqdm

In [ ]:
# Load a dwpose detector
detector = DWposeDetector()
detector = detector.to('cuda')

In [ ]:
# Load Image
ref_image_file = '/cephfs/SZ-AI/usr/liuchenyu/HaiLook/Moore-AnimateAnyone/configs/inference/ref_images/bella_upper.jpg'

H = 512
W = 512

ref_image = cv2.resize(np.array(Image.open(ref_image_file)), (H,W))

ref_pose, score, ref_pose_keypoints = detector(ref_image, output_type='key_points')
score = np.mean(score, axis=-1)

In [ ]:
source = ref_pose_keypoints
source_body = np.array(source["bodies"]["candidate"])
for i in range(len(source_body)):
    if source["bodies"]["subset"][0][i] == -1:
        source_body[i] = np.array([-1., -1.])

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 8))
axs[0].imshow(ref_image)
axs[1].imshow(ref_pose)
axs[0].axis('off')
axs[1].axis('off')

In [ ]:
# Load video
video_file = '/cephfs/SZ-AI/usr/liuchenyu/HaiLook/TikTok_dataset/TikTok_vid/00050.mp4'

driving_vid = read_frames(video_file)

a = Image.fromarray(cv2.resize(np.array(driving_vid[44]), (W, H), interpolation=cv2.INTER_LINEAR))
a

In [ ]:
driving_pose = []
scores = []
vid_key_points = []

for frame in tqdm(driving_vid):
    frame = cv2.resize(np.array(frame), (W, H), interpolation=cv2.INTER_LINEAR)
    pose, score, keypoints = detector(frame, output_type='key_points')
    driving_pose.append(pose)
    score = np.mean(score, axis=-1)
    scores.append(score)
    vid_key_points.append(keypoints)
    

In [ ]:
importlib.reload(openpose_rescaler)
from openpose_rescaler import draw, rescale_skeleton, crop_and_resize

In [ ]:
idx = 122
source = vid_key_points[idx]
targ = ref_pose_keypoints
res = rescale_skeleton(source, targ)
res_pose_img = draw(res, W, H)

fig, axs = plt.subplots(1, 3, figsize=(15, 8))
axs[0].imshow(ref_pose)
axs[1].imshow(driving_pose[idx])
axs[2].imshow(res_pose_img)
axs[0].axis('off')
axs[1].axis('off')
axs[2].axis('off')
plt.show()

In [ ]:
# for i in range(60):
#     idx = i
#     source = vid_key_points[idx]
#     targ = ref_pose_keypoints
#     res = rescale_skeleton(source, targ)
#     res_pose_img = draw(res, W, H)
#     print(i)
#     fig, axs = plt.subplots(1, 3, figsize=(15, 8))
#     axs[0].imshow(ref_pose)
#     axs[1].imshow(driving_pose[idx])
#     axs[2].imshow(res_pose_img)
#     axs[0].axis('off')
#     axs[1].axis('off')
#     axs[2].axis('off')
#     plt.show()

In [ ]:
#videos
processed_pose_vid = []
targ = ref_pose_keypoints
for frame in tqdm(vid_key_points):
    new_frame = rescale_skeleton(frame, targ)
    processed_pose_vid.append(draw(new_frame, H, W))
    # targ = new_frame


In [ ]:
face = vid_key_points[0]["faces"]

In [ ]:
face.shape

In [ ]:
name = "wop_skeleton_rescale_freezeRef"

out_path = '/cephfs/SZ-AI/usr/liuchenyu/HaiLook/Moore-AnimateAnyone/output/exps/' + name + '.mp4'

save_videos_from_pil(processed_pose_vid, out_path, fps=30)

In [ ]:
name = "wop_skeleton"

out_path = '/cephfs/SZ-AI/usr/liuchenyu/HaiLook/Moore-AnimateAnyone/output/exps/' + name + '.mp4'

save_videos_from_pil(driving_pose, out_path, fps=30)